# Creates a  HDF5-Database for a measurement day  

In [46]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
%autoreload 2

In [48]:
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime

''' Sn properties - Masses, Ratios, Cup Configurations, ... '''
from sn_config import *
from collections import Counter
import pandas as pd
import numpy as np

'''Classes for Reading in the Data and applying DS Inversion and Interference corr'''
from nu_data_reduction import NU_data_read, normalisation, evaluation

import pylab as plt
from tables import *

# Define path and parameter of measurement

In [77]:
path = "/Volumes/friebelm/PhD/NU Plasma/Measurements/2018-04-30/"
#path = "/Users/marf/Desktop/PhD Temp/2017-06-12/"
int_norm_hdf = pd.HDFStore(path+"int_norm_20180430_new_bgd.h5")

#path = "/Volumes/friebelm/PhD/NU Plasma/Measurements/2017-12-05/"
files_start = 9442
files_end = 9587

#files_1 = range(files_start, files_end, 1)
#files_1 = range(8005,8069,1) + range(8073,8166, 1)
files_1 = range(9442,9638,1) + range(9639,9749, 1)  + range(9753, 9764)

In [67]:
#ls_rmv = [2101]
#files_1 =[x for x in files_1 if x not in ls_rmv]

In [68]:
# cup configuration
#cup_config = cycle_Sb
#Sn_isotopes = ["112", "114", "115", "116", "117", "118", "119", "120", "122", "124"]
Sn_isotopes = ["117", "118", "119", "120", "122", "124"]

#cup_config = cycles2# 2cycles
cup_config = cycles_spike # 1cycle - 117-129

# Mass Range of cup configuration
#mass_range = cycles2_mass_range # 2 cycles
mass_range = cycle_Sn_spike_mass_range # 1cycle - 117-129

# Isotopes used for Interference correction
#corr_isotopes_2 = {"Cd" : "111", "Te" : "125", "Xe" : "129"}
corr_isotopes_2 = {"Te" : "125", "Xe" : "129"}
#corr_isotopes_Sb = {"Te": "125"}

norm_ratio = ["122","118"]
denom_isotope = norm_ratio[1]
Sn_monitor = ["125", "129"]

def eval_iso_list(isotopes_list, denom_isotope):
    isotope_den = denom_isotope
    isotopes_list_new = [x for x in isotopes_list if x != isotope_den]
    isotopes_list_new.sort()
    return isotopes_list_new

 

# columns for signal dataframe
columns_1 = ["cycle", "sample", "date", "H8 (1)", "H7 (1)", "H6 (1)", "H5 (1)", "H4 (1)", "H3 (1)", 
             "H2 (1)", "H1 (1)", "Ax (1)", "L1 (1)", "L2 (1)", "L3 (1)", "L4 (1)"]


# Amount of interations for estimation of instrumental and natural fractionation
loop_nat = 3
loop_ins = 6
# assumended instrumental and natural fractionation
start_nat = -0.1 #start_nat = -0.1; start_nat = 1000
start_ins = 2 #start_ins = 2; start_ins = 0.001
# isotope ratio used for calculation of instrumental and natural fractionation [denominator[x, y, z]] e.g. [117,[118, 122, 124]] x = 118/117
inv_iso_ratio = 'x'

# Mass fractionation law ("exp" or "GPL")
law = "exp"

# n for GPL
n_GPL_ins = 0.3
n_GPL_nat = 0.001

#number of iterations for beta
iter_beta = 10

# Interference_corr on the denominator isotope
isotope_denom_corr = True

# Methods to evaluate data & extract metadata

In [69]:
#def signals_raw(path, sample, cup_config, columns):
    
def metadata_df(data_read_obj, cycles):
    # extract sample name from csv
    sample_name = df.extract_metadata(sample, "Sample Name")
    # extract date from csv
    date = df.extract_metadata(sample, "Date")
    # extract time from csv
    starttime = df.extract_metadata(sample, "Start Time")
    # extract Number of Analysis Cycles from csv
    Num_of_Ana_Lines = df.extract_metadata(sample, "Number of Analysis Cycles")
    # combine date and starttime
    dateandtime = datetime.strptime(date+starttime, '%d/%m/%Y%H:%M')
    
    df_metadata = pd.DataFrame()
    df_metadata["Cycle"] = cycles
    df_metadata["Filenumber"] = sample
    df_metadata["Sample"] = sample_name
    df_metadata["Date"] = dateandtime
    df_metadata["Num_of_Ana_Cyc"] = Num_of_Ana_Lines
    df_metadata = df_metadata[["Filenumber", "Sample", "Date", "Cycle",  "Num_of_Ana_Cyc"]]
    df_metadata = df_metadata.set_index("Cycle")
    
    return df_metadata

def signals_raw(data_read_obj, sample, cup_config, cycle="cycle1"):
    # create a dataframe with raw signals from csv datafile
    signals_raw_all = df.data_read(sample)
    signals_raw = pd.DataFrame()
    for key in cup_config[cycle]:
        if key in signals_raw_all.columns:
            signals_raw[cup_config[cycle][key]] = signals_raw_all[key]
    
    if cycle == "cycle2":
        signals_raw = signals_raw.add_suffix('_2')
    else:
        None
    
    signals_raw = signals_raw[sorted(signals_raw.columns)]
    return signals_raw

def signals_baseline(data_read_obj, sample, cup_config, cycle="zero1"):
    # create a dataframe with raw signals from csv datafile
    signals_base_all = df.data_read(sample)
    signals_base = pd.DataFrame()
    for key in cup_config[cycle]:
        if key in signals_base_all.columns:
            signals_base[cup_config[cycle][key]] = signals_base_all[key]
    
    signals_base = signals_base[sorted(signals_base.columns)]
    return signals_base

def signals_baseline_corr(data_read_obj, sample, cup_config, cycle="cycle1"):
    # creates a dataframe with all zero corrected signals on the cups

    ## perform baseline correction
    df_zero = df.data_zero_corr(sample)
    # create dataframe with baseline corrected data
    df_zero = pd.DataFrame(df_zero[cycle])
    for key in cup_config[cycle]:
        if key in df_zero.columns:
            df_zero.rename(columns = {key : cup_config[cycle][key]}, inplace=True)
    
    if cycle == "cycle2":
        df_zero = df_zero.add_suffix('_2')
    else:
        None
    
    df_zero = df_zero[sorted(df_zero.columns)]
    df_zero.index.name = "Cycle"
    return df_zero
    
def bgd_before(data_read_obj, blk_ls, path, sample, cup_config):

    cycles = range(1, len(data_read_obj.data_read(sample).index)+1)
        
    # arbitrary blank positions - check blk position before and after sample 
    blk1 = [item for item in blk_ls if item < sample]
    
    if not blk1:
        return None
    else:
        blk1 = blk1[-1]
        # Reads signals of Blank measurements
        blk_1_obj = NU_data_read(path, blk1, cup_config)
        # Substracts baselines of signals
        df_bgd_1 = blk_1_obj.data_zero_corr(blk1)  

        return df_bgd_1

def bgd_after(data_read_obj, blk_ls, path, sample, cup_config):

    cycles = range(1, len(data_read_obj.data_read(sample).index)+1) 
    
    # arbitrary blank positions - check blk position before and after sample 
    blk2 = [item for item in blk_ls if item > sample]
    
    if not blk2:
        return None
    else:
        blk2 = blk2[0]
        # Reads signals of Blank measurements
        blk_2_obj = NU_data_read(path, blk2, cup_config)
        # Substracts baselines of signals
        df_bgd_2 = blk_2_obj.data_zero_corr(blk2)   
    
        return df_bgd_2

def raw_ratios(data_eval_obj):
    #Get raw ratios
    data_sample = data_eval_obj.raw_ratios(denom_isotope)
    # raw ratios from dataframe in dictionary for data handling
    data_sample = pd.DataFrame.from_dict(data_sample, orient = 'index')

    return data_sample

def df_plus_metadata(df_meta, df_values):
    df_values = pd.concat([df_meta, df_values], axis=1, ignore_index=False).reset_index()
    df_values = df_values.set_index("Date")
    
    return df_values

def add_epsilon_Q(df_values):
    #Calculation of eSn, fsam, fspike, Q
    
    
    df_values["eSn_118_117"] = ((df_values["Nr2:x"]/df_values["n0:x"])-1)*10000
    df_values["eSn_122_117"] = ((df_values["Nr2:y"]/df_values["n0:y"])-1)*10000
    df_values["eSn_124_117"] = ((df_values["Nr2:z"]/df_values["n0:z"])-1)*10000
    df_values["eSn_122_118"] = (((df_values["Nr2:y"]/df_values["Nr2:x"])/(df_values["n0:y"]/df_values["n0:x"]))-1)*10000
    df_values["eSn_124_118"] = (((df_values["Nr2:z"]/df_values["Nr2:x"])/(df_values["n0:z"]/df_values["n0:x"]))-1)*10000
    df_values["fsam"] = 1/((df_values["Mr2.5:x"]*0.07669971 - 0.24220048)/(0.00525699 - df_values["Mr2.5:x"]*0.51062751)+1)
    df_values["fspike"] = 1-df_values["fsam"]
    df_values["Q"] = df_values["fsam"]/df_values["fspike"]
    
    return df_values

# Database files for raw data

In [78]:
# Create Database for raw data

blk_ls = []
sample_ls = []

for sample in files_1:
    # read data from csv file
    df = NU_data_read(path, sample, cup_config)
    sample_name = df.extract_metadata(sample, "Sample Name")

    # determine number of cycles of file
    cycles = range(1, len(df.data_read(sample).index)+1)

    df_meta = metadata_df(df, cycles)

    ### baseline corrected signals
    df_signals_1 = signals_raw(df, sample, cup_config, "cycle1")
    df_signals = df_plus_metadata(df_meta, df_signals_1)
    #df_signals = df_signals.set_index("Date")
    int_norm_hdf.append('/raw_data/signals_raw', df_signals, min_itemsize = {"Sample": 50})
    
    df_baseline = signals_baseline(df, sample, cup_config)
    df_baseline = df_plus_metadata(df_meta, df_baseline)
    int_norm_hdf.append('/raw_data/baselines', df_baseline, min_itemsize = {"Sample": 50})
    
    df_signals_zero_1 = signals_baseline_corr(df, sample, cup_config, "cycle1")
    df_signals_zero = df_plus_metadata(df_meta, df_signals_zero_1)
    int_norm_hdf.append('/raw_data/signals_zero', df_signals_zero, min_itemsize = {"Sample": 50})
    
    
    # create list with sample file numbers of sample measurements and blank measurements
    if sample_name == "blank sol" or sample_name == "wash" or sample_name == "wash clean" or sample_name == "Teflon blk":
        blk_ls.append(sample)
    elif sample_name == "SQ" or sample_name == "Teflon blk" or sample_name == "4ml HDPE blank":
        None
    else:
        sample_ls.append(sample)

In [79]:
signals = int_norm_hdf['/raw_data/signals_zero']

In [80]:
signals[(signals["118"] < 0.1) & (signals["Sample"] != "blank sol")]["Filenumber"].unique()

array([], dtype=int64)

# Database for internal norm. - 1cycle

In [81]:
# Dictionary to loop over for different interference corrections
interf_corr_dict = {"No_interference_corr" : {}, "Te125_Xe129_corr" : {"Cd": "111", "Te" : "125", "Xe" : "129"}, "Te126_Xe129_corr" : {"Cd": "111", "Te" : "126", "Xe" : "129"}}

# Dictionary to loop over different normalisations
norm_dict = [["122","117"],["118","120"], ["122","118"]]

# List to loop over for bgd_corr and no_bgd_cor
#bgd_corr_ls = [True, False]

# background correction
bgd_corr = True
# Outlier rejection un-/activated for background data
bgd_outlier_rej = True

for sample in sample_ls:
    # read data from csv file
    data_raw_obj = NU_data_read(path, sample, cup_config)
    # baseline corrected dataframe
    df_zero = data_raw_obj.data_zero_corr(sample)
    # determine number of cycles of file
    cycles = range(1, len(data_raw_obj.data_read(sample).index)+1)
    # dataframe with metadata(e.g. Filenumber, Date, etc.)
    df_meta = metadata_df(data_raw_obj, cycles)

    for norm_ratio in norm_dict:
        denom_isotope = norm_ratio[1]
        isotope_ls = eval_iso_list(Sn_isotopes, denom_isotope)
        isotopes = [isotope_ls]
        
        data_sample_column = [(i + "/" + denom_isotope) for i in isotopes[0]]
        norm_key = 'r_' + norm_ratio[0] + "_" + norm_ratio[1]
        
        # iterate over type of interference correction  
        for key in interf_corr_dict:

            # create Data_evaluation object
            data_eval_obj = evaluation(df_zero, cycles, isotopes, cup_config, database, mass_range, interf_corr_dict[key], denom_corr_ratio, law, n_GPL_ins)

            # perform on peak background correction
            if bgd_corr == True:
                bgd1 = bgd_before(df, blk_ls, path, sample, cup_config)
                bgd2 = bgd_after(df, blk_ls, path, sample, cup_config)
                if bgd_outlier_rej == True:
                    bgd1 = data_eval_obj.mad_outlier_rejection_dict(bgd1)
                    bgd2 = data_eval_obj.mad_outlier_rejection_dict(bgd2)
                    bgd_method = 'bgd_outlier_corr'
                else:
                    bgd_method = 'bgd_no_outlier_corr'
                
                df_bgd_corr = pd.DataFrame(df_zero['cycle1']) - pd.DataFrame(data_eval_obj.data_bgd_corr_2(bgd1, bgd2)['cycle1'])
                for cup in cup_config['cycle1']:
                    if cup in df_bgd_corr.columns:
                        df_bgd_corr.rename(columns = {cup : cup_config['cycle1'][cup]}, inplace=True)
    
                df_bgd_corr = df_bgd_corr[sorted(df_bgd_corr.columns)]
                
            else:
                df_bgd_corr = pd.DataFrame()
                bgd_method = 'no_bgd_corr'
            
            #line2 correction
            data_eval_obj.line2_corr(data_eval_obj.get_df(), "119")
            
            # get raw ratios for DS-inversion
            df_raw_ratios = raw_ratios(data_eval_obj)
            
            # internal_normalisation
            data_int_norm = data_eval_obj.internal_norm_1(norm_ratio, denom_isotope, iter_beta)
            data_int_norm = pd.DataFrame.from_dict(data_int_norm, orient = 'index')
            data_int_norm = data_int_norm[sorted(data_int_norm.columns)]
            data_int_norm.columns = data_sample_column

            # Interference correction
            beta = data_eval_obj.beta("Sn", norm_ratio, iter_beta)
            raw_ratios_corr = data_eval_obj.norm_beta_to_raw("Sn", denom_isotope, beta)

            # interference corrected raw ratios
            df_corr_raw_ratios = pd.DataFrame.from_dict(raw_ratios_corr, orient = 'index')
    
            # amount of Interference correction
            df_inter_corr = df_raw_ratios - df_corr_raw_ratios
            df_raw_ratios = df_raw_ratios[sorted(df_raw_ratios.columns)]
            df_raw_ratios.columns = data_sample_column
            df_inter_corr = df_inter_corr[sorted(df_inter_corr.columns)]
            df_inter_corr.columns = data_sample_column
            df_corr_raw_ratios = df_corr_raw_ratios[sorted(df_corr_raw_ratios.columns)]
            df_corr_raw_ratios.columns = data_sample_column
            
            df_raw_ratios = df_plus_metadata(df_meta, df_raw_ratios)
            
            df_corr_raw_ratios = df_plus_metadata(df_meta, df_corr_raw_ratios)
            int_norm_hdf.append('/evaluation/'+bgd_method +'/' + norm_key +'/' +key +'/df_raw_ratios_af_interf_corr', df_corr_raw_ratios, min_itemsize = {"Sample": 50})
            
            data_int_norm = df_plus_metadata(df_meta, data_int_norm)
            int_norm_hdf.append('/evaluation/'+bgd_method +'/' + norm_key +'/' +key +'/df_internal_norm', data_int_norm, min_itemsize = {"Sample": 50})
            
            df_inter_corr = df_plus_metadata(df_meta, df_inter_corr) 
            int_norm_hdf.append('/evaluation/'+bgd_method +'/' + norm_key +'/' +key  +'/df_amount_interf_corr', df_inter_corr, min_itemsize = {"Sample": 50})

    df_bgd_corr = df_plus_metadata(df_meta, df_bgd_corr)
    int_norm_hdf.append('/evaluation/'+bgd_method  +'/df_bgd_corr' , df_bgd_corr, min_itemsize = {"Sample": 50})

# Close Database after adding all Data

In [82]:
df_bgd_corr

,Cycle,Filenumber,Sample,Num_of_Ana_Cyc,117,118,119,120,121,122,123,124,125,126,127,129,131
Date,,,,,,,,,,,,,,,,,
2018-05-03 12:06:00,1,9762,NIST 200ppb,1,0.001132,0.003521,0.001258,0.00677,0.00008,0.000689,-0.000356,0.000931,0.000009,0.000089,0.080901,0.017953,0.014755
2018-05-03 12:06:00,2,9762,NIST 200ppb,1,0.001132,0.003521,0.001258,0.00677,0.00008,0.000689,-0.000356,0.000931,0.000009,0.000089,0.080901,0.017953,0.014755
2018-05-03 12:06:00,3,9762,NIST 200ppb,1,0.001132,0.003521,0.001258,0.00677,0.00008,0.000689,-0.000356,0.000931,0.000009,0.000089,0.080901,0.017953,0.014755
2018-05-03 12:06:00,4,9762,NIST 200ppb,1,0.001132,0.003521,0.001258,0.00677,0.00008,0.000689,-0.000356,0.000931,0.000009,0.000089,0.080901,0.017953,0.014755
2018-05-03 12:06:00,5,9762,NIST 200ppb,1,0.001132,0.003521,0.001258,0.00677,0.00008,0.000689,-0.000356,0.000931,0.000009,0.000089,0.080901,0.017953,0.014755
2018-05-03 12:06:00,6,9762,NIST 200ppb,1,0.001132,0.003521,0.001258,0.00677,0.00008,0.000689,-0.000356,0.000931,0.000009,0.000089,0.080901,0.017953,0.014755
2018-05-03 12:06:00,7,9762,NIST 200ppb,1,0.001132,0.003521,0.001258,0.00677,0.00008,0.000689,-0.000356,0.000931,0.000009,0.000089,0.080901,0.017953,0.014755
2018-05-03 12:06:00,8,9762,NIST 200ppb,1,0.001132,0.003521,0.001258,0.00677,0.00008,0.000689,-0.000356,0.000931,0.000009,0.000089,0.080901,0.017953,0.014755
2018-05-03 12:06:00,9,9762,NIST 200ppb,1,0.001132,0.003521,0.001258,0.00677,0.00008,0.000689,-0.000356,0.000931,0.000009,0.000089,0.080901,0.017953,0.014755


In [83]:
int_norm_hdf['/evaluation/'+bgd_method +'/' + norm_key +'/' +key +'/df_internal_norm']

,Cycle,Filenumber,Sample,Num_of_Ana_Cyc,117/118,119/118,120/118,122/118,124/118
Date,,,,,,,,,
2018-04-30 22:18:00,1,9443,NIST 8 ppb,1,0.316661,0.354633,1.345912,0.19125,0.238848
2018-04-30 22:18:00,2,9443,NIST 8 ppb,1,0.316588,0.354681,1.346191,0.19125,0.239031
2018-04-30 22:18:00,3,9443,NIST 8 ppb,1,0.316569,0.354655,1.346307,0.19125,0.239130
2018-04-30 22:18:00,4,9443,NIST 8 ppb,1,0.316607,0.354673,1.346077,0.19125,0.239014
2018-04-30 22:18:00,5,9443,NIST 8 ppb,1,0.316564,0.354655,1.346293,0.19125,0.239099
2018-04-30 22:18:00,6,9443,NIST 8 ppb,1,0.316581,0.354605,1.346019,0.19125,0.238991
2018-04-30 22:18:00,7,9443,NIST 8 ppb,1,0.316561,0.354560,1.345847,0.19125,0.238942
2018-04-30 22:18:00,8,9443,NIST 8 ppb,1,0.316456,0.354635,1.346232,0.19125,0.239152
2018-04-30 22:18:00,9,9443,NIST 8 ppb,1,0.316564,0.354606,1.346060,0.19125,0.239012


In [84]:
int_norm_hdf.close()

In [85]:
Te_ratios.get_all_abundances("128")

{'120': 0.09268988327552521,
 '122': 2.5277124968762017,
 '123': 0.8860632075780251,
 '124': 4.716419446013621,
 '125': 7.050686664056489,
 '126': 18.806742387220165,
 '128': 31.753985363317998,
 '130': 34.165700551661985}